In [121]:
# tratamiento de los datos
# ============================================
import pandas as pd
import numpy as np

# librerías para la visualización de los datos
# ============================================
import matplotlib.pyplot as plt
import seaborn as sns

# Establecer tamaño fijo de gráficas
# ==================================
plt.rcParams["figure.figsize"] = (10,8)

# Configuración warnings
# ======================
import warnings
warnings.filterwarnings('ignore')

# Pair Programming EDA
### Ana Gonzalez y Ana Campos

Para los ejercicios de pair programming de Regresión logística temdremos que buscar un dataset (al igual que hicismos en regresión lineal) que usaremosmos a lo largo de los siguientes ejercicios.
Se ruega a la hora de realizar la entrega que incluyais el conjunto de datos que hayais decidido emplear para estos ejercicios.
## Objetivos
Buscar un **conjunto** de datos a analizar
Se recomienda que el conjunto de datos a analizar tenga variables **numéricas** y categóricas, primando que haya más de una variable de tipo numérico.
- Explicar los **datos** y las **variables disponibles** en el conjunto de datos seleccionado
- Realizar un **EDA** sencillo poniendo en práctica los conocimientos adquiridos hasta el momento.
- Interpretación de los **resultados**.

In [ ]:
df = pd.read_csv("data-log/.csv", index_col = 0).reset_index()
df.head(2)

## Entender las variables que tenemos y como es nuetro dataframe. 

In [ ]:
# número de filas y columnas del dataframe
df.shape

In [ ]:
# información general del dataframe
df.info()

In [ ]:
# nombre de las columnas de nuestro dataframe
df.columns

In [ ]:
# principales estadísticos de las columnas numéricas. recordemos que por defecto el describe nos devuelve las numéricas
df.describe().T

In [ ]:
# principales estadisticos de las variables categóricas
df.describe(include ="object").T

## Nulos

In [ ]:
# valores nulos en el dataframe
df.isnull().sum()

## Duplicados

In [ ]:
# duplicados 
df.duplicated().sum()

In [ ]:
df[df.duplicated()== True] # Para ver la fila duplicada

In [131]:
df.drop_duplicates(inplace=True)

In [ ]:
df.duplicated().sum()

In [ ]:
# duplicados 
df.duplicated().sum()

In [ ]:
df.shape

# Outliers

In [ ]:
df.describe().T

In [ ]:
numericas = df.select_dtypes(include=np.number)
numericas.head(2)

In [ ]:
fig, axes = plt.subplots(2,2, figsize=(20,10))
axes = axes.flat
for indice, columna in enumerate(numericas.columns):
    sns.boxplot(x = numericas[columna], data = df, ax=axes[indice], color = "turquoise"); 
plt.tight_layout()
plt.show();

Detectar valores de los outliers

In [157]:
def detectar_outliers(lista_columnas, dataframe): 
    dicc_indices = {} # creamos un diccionario donde almacenaremos índices de los outliers
    # iteramos por la lista de las columnas numéricas de nuestro dataframe
    for col in lista_columnas:
        #calculamos los cuartiles Q1 y Q3
        Q1 = np.nanpercentile(dataframe[col], 25)
        Q3 = np.nanpercentile(dataframe[col], 75)
        # calculamos el rango intercuartil
        IQR = Q3 - Q1
        # calculamos los límites
        outlier_step = 1.5 * IQR
        # filtramos nuestro dataframe para indentificar los outliers
        outliers_data = dataframe[(dataframe[col] < Q1 - outlier_step) | (dataframe[col] > Q3 + outlier_step)]
        if outliers_data.shape[0] > 0: # chequeamos si nuestro dataframe tiene alguna fila. 
            dicc_indices[col] = (list(outliers_data.index)) 
    return dicc_indices 

In [158]:
#lista_columnas = list(numericas.columns)
#ind = detectar_outliers(lista_columnas, df)

In [159]:
# en que columnas nos hemos encontrado con outliers? 
#ind.keys()

In [160]:
# Iteramos por el diccionario que hemos creado con los índices de los outliers
#for k, v in ind.items():
    # Sacamos la mediana de cada columna numérica
    #mediana = df[k].median()
    # iteramos por cada fila de la columna en la que hay outliers y convertimos ese valor en la mediana
    #for i in v:
        #df.loc[i,k] = mediana



In [ ]:
df.describe().T

## Analizar las relaciones entre las variables. 

### Variable respuesta

In [ ]:
df.head(2)

In [ ]:
# vamos a ver visualmente como está distribuida la variable respuesta
sns.set(style="ticks")
sns.set_style("darkgrid")
sns.distplot(
    df["charges"], 
    #hist = False, 
    color = "blue", 
    kde_kws = {"shade": True, "linewidth": 1});

In [ ]:
df.skew()

#### Variables numéricas

In [ ]:
# exploramos cuáles son los principales estadísticos de las variables numéricas

df.select_dtypes(include = np.number).describe().T

In [ ]:
# Gráfico de relación entre variables predictoras y la variable a predecir
# ==============================================================================

fig, axes = plt.subplots(nrows = 2, ncols = 3, figsize = (30, 10))
axes = axes.flat

columnas_numeric = df.select_dtypes(include = np.number).columns
columnas_numeric = columnas_numeric.drop("charges")

for i, colum in enumerate(columnas_numeric):
    sns.regplot(
        x = df[colum], 
        y = df["charges"], 
        color = "blue", 
        marker = ".", 
        scatter_kws = {"alpha": 0.4}, 
        line_kws = {"color": "red", "alpha": 0.7 }, 
        ax = axes[i])
    
    axes[i].set_title(f"Prima del seguro vs {colum}", fontsize = 10, fontweight = "bold")
    axes[i].tick_params(labelsize = 20)
    axes[i].set_xlabel("")
    axes[i].set_ylabel("")
    
fig.tight_layout()
fig.delaxes(axes[-1])
fig.delaxes(axes[-2])
fig.delaxes(axes[-3]);

In [ ]:
plt.figure(figsize = (10,5))  # establecemos el tamaño de la gráfica
sns.histplot(data =df3, x = "charges", hue = "smoker") 
plt.xticks(rotation = 45) # rotamos las etiquetas del eje x para que estén a 45 grados

#### Correlación entre variables numéricas

In [ ]:
sns.heatmap(df.corr(), 
           cmap = "YlGnBu", 
           annot = True);

In [ ]:
# creamos un dataframe todo de unos de la misma forma que nuestra matriz de correlación

mask = np.triu(np.ones_like(df.corr(), dtype = np.bool))

sns.heatmap(df.corr(), 
           cmap = "YlGnBu", 
            mask = mask,
           annot = True);

#### Variables categóricas

In [ ]:
df.head(2)

In [ ]:
categoricas = df.select_dtypes(include = 'object').describe().T
categoricas.head()

In [ ]:
sns.barplot(x='region', y='charges', hue='sex', data=df, palette='Set2');

In [ ]:
df.to_csv("../data-log/01-dataframe.csv" )